In [ ]:
#avoidance controller 1
import time
import sys
import urx
import numpy as np
import keyboard
import json
import numpy as np
from numpy.linalg import inv
import time

interval = True


try:
    rob = urx.Robot("192.168.88.249")
except:
    print('robot error')
    interval = False


if interval:

    #Controller
    dt = 0.1


    print('1')
    P = np.array([0.2,0.2,0.2]).reshape(3,1);
    P_obs = np.array([0.2,0.2,0.2]).reshape(3,1);
    V = np.array([0.0,0.0,0.0]).reshape(3,1)
    Po = np.array([2.1,2.1,10.1]).reshape(3,1);
    e = np.zeros((3,1));
    ea = np.zeros((3,1));
    lamda = 0.01
    k2 = (np.diag([1.2,2.3,3.3]));
    k1 = ( np.diag([1.1,2.3,1.3]));
    k3 = ( np.diag([1.1,2.3,1.3]));

        
    while True:
        tic = time.clock()
        
        try:
            with open('data2.json', 'r') as fa:
                data = json.load(fa)
                k1[0,0] = data['controller']['k11']
                k1[1,1] = data['controller']['k12']
                k1[2,2] = data['controller']['k13']
                k2[0,0] = data['controller']['k21']
                k2[1,1] = data['controller']['k22']
                k2[2,2] = data['controller']['k23']
                k3[0,0] = data['controller']['k31']
                k3[1,1] = data['controller']['k32']
                k3[2,2] = data['controller']['k33']
                vmax = data['controller']['vmax']
                x_refr = data['poss']['xref']
                y_refr = data['poss']['yref']
                z_refr = data['poss']['zref']
                xo_refr = data['poss_obs']['xref']
                yo_refr = data['poss_obs']['yref']
                zo_refr = data['poss_obs']['zref']
                dmin = data['poss_obs']['dmin']
                kd1 = data['poss_obs']['kd1']
                kd2 = data['poss_obs']['kd2']
                kd3 = data['poss_obs']['kd3']
                tau = data['poss_obs']['tau']
        except:
            print('file error')


        t = rob.get_pose()
      #  print("Transformation from base to tcp is: ", t)


        pose = rob.getl()
        xn = float(pose[0])
        yn = float(pose[1])
        zn = float(pose[2])
        
        P = np.zeros((3,1));
        P[0,0] = float(xn)
        P[1,0] = float(yn)
        P[2,0] = float(zn)
    #    print('Actual Point',P)
        
        Pa = np.zeros((3,1));   
        Pa[0,0] = float(x_refr)
        Pa[1,0] = float(y_refr)
        Pa[2,0] = float(z_refr)
     #   print('Desired Point',Po)
        
        P_obs = np.zeros((3,1));   
        P_obs[0,0] = float(xo_refr)
        P_obs[1,0] = float(yo_refr)
        P_obs[2,0] = float(zo_refr)
     #   print('Obstacle point',P_obs)
    
        e = Pa - P      
        
        hv = np.matmul(k2,np.tanh(np.matmul(k1,e) + np.matmul(k3,e-ea)))
        ea = e
        
        r_ao = P_obs - P
        n_r_ao = np.linalg.norm(r_ao)
        
        if n_r_ao < dmin:
            #First collision condition
            print('collision condition 1')
            print('e')
            ang = np.arccos(np.matmul(V.transpose(),r_ao)/ (np.linalg.norm(V)*np.linalg.norm(r_ao)) )
            
            if ang < (90)*np.pi/180:
                #seocnd collision condition
                nr = P / np.linalg.norm(P)
                print('collision condition 2')
                nrg = e / np.linalg.norm(e)
                #print(np.linalg.norm(e))
                nv = V / np.linalg.norm(V)
                #print('nv',np.linalg.norm(nv),nv)
                nro = r_ao / np.linalg.norm(r_ao)
                #print('nro',np.linalg.norm(nro),nro)
                a = np.cross(nro.flatten(),nv.flatten())
                #print('a',np.linalg.norm(a),a)
                b = np.cross(a.flatten(),nro.flatten())
                b = b /np.linalg.norm(b)
                #print('b',np.linalg.norm(b),b)
                P_aux = np.zeros((3,1)) 
                P_aux[0,0] = -float(xn)
                P_aux[1,0] = -float(yn) 
                ang_aux = np.arccos(np.matmul(b,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b)) )
                if ang_aux > np.pi/2 :
                    b = b
                else :
                    b = -b
                a1 = np.cross(nro.flatten(),P.flatten())
                #print('a',np.linalg.norm(a1))
                b1 = np.cross(a1.flatten(),nro.flatten())
                #print('b',np.linalg.norm(b1))
                b1 = b1 /np.linalg.norm(b1)
                ang_aux = np.arccos(np.matmul(b1,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b1)) )
                if ang_aux > np.pi/2 :
                    b1 = b1
                else :
                    b1 = -b1
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Fd = np.matmul(V.reshape([1,3]),ur.reshape([3,1]))
                Ft = kd1*b.reshape([3,1])*Fk + kd2*b1.reshape([3,1])*Fk - kd3*Fk*ur.reshape([3,1]) 
                #print('Ft',Ft)
                #print('b',b)
                #print('b1',b1)
                #print('ur',ur)
                #print('hv',hv)
                vx1 = np.exp(-dmin*tau)*Ft[0,0] + (1-np.exp(-dmin*tau))*hv[0,0]
                vy1 = np.exp(-dmin*tau)*Ft[1,0] + (1-np.exp(-dmin*tau))*hv[1,0]
                vz1 = np.exp(-dmin*tau)*Ft[2,0] + (1-np.exp(-dmin*tau))*hv[2,0]
            else:                  
                vx1 = hv[0,0]
                vy1 = hv[1,0]
                vz1 = hv[2,0]
        else:
            vx1 = hv[0,0]
            vy1 = hv[1,0]
            vz1 = hv[2,0]
        
        VMAX = np.sqrt(vx1*vx1+vy1*vy1+vz1*vz1) 
        
        if (VMAX > vmax):
            vx1 = vx1*vmax/VMAX
            vy1 = vy1*vmax/VMAX
            vz1 = vz1*vmax/VMAX
     



        V[0,0] = vx1
        V[1,0] = vy1
        V[2,0] = vz1


        try:
            rob.speedl((vx1 , vy1, vz1 , 0, 0, 0), acc=0.3, min_time=3);
        except:
            print('sent')

        time.sleep(0)

        toc = time.clock()
        print(e)

else:
    print('finishing')
    try:
        rob.stopl()
    except:
        print('No')

In [ ]:
#avoidance controller 2
import time
import sys
import urx
import numpy as np
import keyboard
import json
import numpy as np
from numpy.linalg import inv
import time

interval = True
f= open("paper8.txt","w+")

#v =triad_openvr.triad_openvr()

try:
    rob = urx.Robot("192.168.88.240")
except:
    print('robot error')
    interval = False


if interval:

    #Controller
    dt = 0.1


    print('1')
    P = np.array([0.2,0.2,0.2]).reshape(3,1);
    P_obs = np.array([0.2,0.2,0.2]).reshape(3,1);
    V = np.array([0.0,0.0,0.0]).reshape(3,1)
    Po = np.array([2.1,2.1,10.1]).reshape(3,1);
    e = np.zeros((3,1));
    ea = np.zeros((3,1));
    lamda = 0.01
    k2 = (np.diag([1.2,2.3,3.3]));
    k1 = ( np.diag([1.1,2.3,1.3]));
    k3 = ( np.diag([1.1,2.3,1.3]));

        
    while True:
        tic = time.clock()
        
        try:
            with open('data2.json', 'r') as fa:
                data = json.load(fa)
                k1[0,0] = data['controller']['k11']
                k1[1,1] = data['controller']['k12']
                k1[2,2] = data['controller']['k13']
                k2[0,0] = data['controller']['k21']
                k2[1,1] = data['controller']['k22']
                k2[2,2] = data['controller']['k23']
                k3[0,0] = data['controller']['k31']
                k3[1,1] = data['controller']['k32']
                k3[2,2] = data['controller']['k33']
                vmax = data['controller']['vmax']
                vmax2 = data['poss']['vmax']
                x_refr = data['poss']['xref']
                y_refr = data['poss']['yref']
                z_refr = data['poss']['zref']
                xo_refr = data['poss_obs']['xref']
                yo_refr = data['poss_obs']['yref']
                zo_refr = data['poss_obs']['zref']
                dmin = data['poss_obs']['dmin']
                kd1 = data['poss_obs']['kd1']
                kd2 = data['poss_obs']['kd2']
                kd3 = data['poss_obs']['kd3']
                tau = data['poss_obs']['tau']
                kangle = data['poss_obs']['kangle']
        except:
            print('file error')


        t = rob.get_pose()
      #  print("Transformation from base to tcp is: ", t)


        pose = rob.getl()
        xn = float(pose[0])
        yn = float(pose[1])
        zn = float(pose[2])
        
        P = np.zeros((3,1));
        P[0,0] = float(xn)
        P[1,0] = float(yn)
        P[2,0] = float(zn)
    #    print('Actual Point',P)
        
        Pa = np.zeros((3,1));   
        Pa[0,0] = float(x_refr)
        Pa[1,0] = float(y_refr)
        Pa[2,0] = float(z_refr)
     #   print('Desired Point',Po)
        
        P_obs = np.zeros((3,1));   
        P_obs[0,0] = float(xo_refr)
        P_obs[1,0] = float(yo_refr)
        P_obs[2,0] = float(zo_refr)
     #   print('Obstacle point',P_obs)
    
        e = Pa - P      
        
        hv = np.matmul(k2,np.tanh(np.matmul(k1,e) + np.matmul(k3,e-ea)))
        ea = e
        
        r_ao = P_obs - P
        n_r_ao = np.linalg.norm(r_ao)
        
        if n_r_ao < dmin:
            #First collision condition
            print('collision condition 1')
            teta = np.arccos(np.matmul(r_ao.transpose(),e)/ (np.linalg.norm(r_ao)*np.linalg.norm(e)) )
            alfa = np.arccos(np.matmul(r_ao.transpose(),-P)/ (np.linalg.norm(r_ao)*np.linalg.norm(P)) ) 
            beta = np.arccos(np.matmul(e.transpose(),-P)/ (np.linalg.norm(P)*np.linalg.norm(e)) )
            print('r_ao',r_ao)
            print('v',V)
            
            print('ang',alfa, beta+teta)
            if  teta < np.pi*kangle/180:
                #seocnd collision condition
                nr = P / np.linalg.norm(P)
                print('collision condition 2')
                nrg = e / np.linalg.norm(e)
                #print(np.linalg.norm(e))
                nv = V / np.linalg.norm(V)
                #print('nv',np.linalg.norm(nv),nv)
                nro = r_ao / np.linalg.norm(r_ao)
                
                
                # SPEED
                
                #print('nro',np.linalg.norm(nro),nro)
                a = np.cross(nro.flatten(),nv.flatten())
                #print('a',np.linalg.norm(a),a)
                b = np.cross(a.flatten(),nro.flatten())
                b = b /np.linalg.norm(b)
                #print('b',np.linalg.norm(b),b)
                P_aux = np.zeros((3,1)) 
                P_aux[0,0] = -float(xn)
                P_aux[1,0] = -float(yn) 
                ang_aux = np.arccos(np.matmul(b,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b = b
                else :
                    b = -b
  
                
                # AXIS Z
                
                a1 = np.cross(nro.flatten(),P_aux.flatten())
                #print('a',np.linalg.norm(a1))
                b1 = np.cross(a1.flatten(),nro.flatten())
                #print('b',np.linalg.norm(b1))
                b1 = b1 /np.linalg.norm(b1)
                ang_aux = np.arccos(np.matmul(b1,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b1)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b1 = b1
                else :
                    b1 = -b1
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Fd = np.matmul(V.reshape([1,3]),ur.reshape([3,1]))
                Ft = kd1*b.reshape([3,1])*Fk + kd2*b1.reshape([3,1])*Fk - kd3*Fk*ur.reshape([3,1]) 
                #print('Ft',Ft)
                print('P',P)
                print('nro',nro)
                print('b',b)
                print('b1',b1)
                print('ur',ur)
                #print('hv',hv)
                vx1 = np.exp(-dmin*tau)*Ft[0,0] + (1-np.exp(-dmin*tau))*hv[0,0]
                vy1 = np.exp(-dmin*tau)*Ft[1,0] + (1-np.exp(-dmin*tau))*hv[1,0]
                vz1 = np.exp(-dmin*tau)*Ft[2,0] + (1-np.exp(-dmin*tau))*hv[2,0]
            else:                  
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Ft = - kd3*Fk*ur.reshape([3,1]) 
                vx1 = 0.8*np.exp(-dmin*tau)*Ft[0,0] + 1.2*(1-np.exp(-dmin*tau))*hv[0,0]
                vy1 = 0.8*np.exp(-dmin*tau)*Ft[1,0] + 1.2*(1-np.exp(-dmin*tau))*hv[1,0]
                vz1 = 0.8*np.exp(-dmin*tau)*Ft[2,0] + 1.2*(1-np.exp(-dmin*tau))*hv[2,0]
        else:
            vx1 = hv[0,0]
            vy1 = hv[1,0]
            vz1 = hv[2,0]
        
        
        
        VMAX = np.sqrt(vx1*vx1+vy1*vy1+vz1*vz1) 
        
        if (VMAX > vmax):
            vx1 = vx1*vmax/VMAX
            vy1 = vy1*vmax/VMAX
            vz1 = vz1*vmax/VMAX



        V[0,0] = vx1
        V[1,0] = vy1
        V[2,0] = vz1


        try:
            
            
            rob.speedl((vx1 , vy1, vz1 , 0, 0, 0), acc=0.3, min_time=3);
        except:
            print('sent')

        time.sleep(0)

        toc = time.clock()
        print(e)
        f.write(str(xn)+";"+str(yn)+";"+str(zn)+";"+str(tic)+";"+str(dmin)+";"+str(kd1)+";"+str(kd2)+";"+str(kd3)+";"+str(tau)+";"+str(vmax)+";"+str(kangle)+"\n")
            
else:
    print('finishing')
    try:
        rob.stopl()
    except:
        print('No')

In [ ]:
#avoidance controller 2 + trajectory
import time
import sys
import urx
import numpy as np
import keyboard
import json
import numpy as np
from numpy.linalg import inv
import time

interval = True
f= open("paper8.txt","w+")

#v =triad_openvr.triad_openvr()

try:
    rob = urx.Robot("192.168.88.240")
except:
    print('robot error')
    interval = False


if interval:

    #Controller
    dt = 0.1


    print('1')
    P = np.array([0.2,0.2,0.2]).reshape(3,1);
    P_obs = np.array([0.2,0.2,0.2]).reshape(3,1);
    V = np.array([0.0,0.0,0.0]).reshape(3,1)
    Po = np.array([2.1,2.1,10.1]).reshape(3,1);
    e = np.zeros((3,1));
    ea = np.zeros((3,1));
    lamda = 0.01
    k2 = (np.diag([1.2,2.3,3.3]));
    k1 = ( np.diag([1.1,2.3,1.3]));
    k3 = ( np.diag([1.1,2.3,1.3]));

    to = time.clock()
        
    while True:
        tic = time.clock()
        
        try:
            with open('data2.json', 'r') as fa:
                data = json.load(fa)
                k1[0,0] = data['controller']['k11']
                k1[1,1] = data['controller']['k12']
                k1[2,2] = data['controller']['k13']
                k2[0,0] = data['controller']['k21']
                k2[1,1] = data['controller']['k22']
                k2[2,2] = data['controller']['k23']
                k3[0,0] = data['controller']['k31']
                k3[1,1] = data['controller']['k32']
                k3[2,2] = data['controller']['k33']
                vmax = data['controller']['vmax']
                vmax2 = data['poss_obs']['vmax']
                x_refr = data['poss']['xref']
                y_refr = data['poss']['yref']
                z_refr = data['poss']['zref']
                T = data['poss']['T']
                xo_refr = data['poss_obs']['xref']
                yo_refr = data['poss_obs']['yref']
                zo_refr = data['poss_obs']['zref']
                dmin = data['poss_obs']['dmin']
                kd1 = data['poss_obs']['kd1']
                kd2 = data['poss_obs']['kd2']
                kd3 = data['poss_obs']['kd3']
                tau = data['poss_obs']['tau']
                kangle = data['poss_obs']['kangle']
        except:
            print('file error')


        t = rob.get_pose()
      #  print("Transformation from base to tcp is: ", t)

        
        pose = rob.getl()
        xn = float(pose[0])
        yn = float(pose[1])
        zn = float(pose[2])
        
        P = np.zeros((3,1));
        P[0,0] = float(xn)
        P[1,0] = float(yn)
        P[2,0] = float(zn)
    #    print('Actual Point',P)
        
        A = 0.3
        Pa = np.zeros((3,1));   
        tact = tic - to;
        y_refr = (A * np.cos(2*np.pi*(tact)/T))
        z_refr = (A * np.sin(2*np.pi*(tact)/T) + 0.5)
        Pa[0,0] = float(x_refr)
        Pa[1,0] = float(y_refr)
        Pa[2,0] = float(z_refr)
        print('Desired Point',Pa)
        
        P_obs = np.zeros((3,1));   
        P_obs[0,0] = float(xo_refr)
        P_obs[1,0] = float(yo_refr)
        P_obs[2,0] = float(zo_refr)
     #   print('Obstacle point',P_obs)
    
        e = Pa - P      
        
        hv = np.matmul(k2,np.tanh(np.matmul(k1,e) + np.matmul(k3,e-ea)))
        ea = e
        
        r_ao = P_obs - P
        n_r_ao = np.linalg.norm(r_ao)
        
        if n_r_ao < dmin:
            #First collision condition
            print('collision condition 1')
            teta = np.arccos(np.matmul(r_ao.transpose(),e)/ (np.linalg.norm(r_ao)*np.linalg.norm(e)) )
            alfa = np.arccos(np.matmul(r_ao.transpose(),-P)/ (np.linalg.norm(r_ao)*np.linalg.norm(P)) ) 
            beta = np.arccos(np.matmul(e.transpose(),-P)/ (np.linalg.norm(P)*np.linalg.norm(e)) )
            print('r_ao',r_ao)
            print('v',V)
            
            print('ang',alfa, beta+teta)
            if  teta < np.pi*kangle/180:
                #seocnd collision condition
                nr = P / np.linalg.norm(P)
                print('collision condition 2')
                nrg = e / np.linalg.norm(e)
                #print(np.linalg.norm(e))
                nv = V / np.linalg.norm(V)
                #print('nv',np.linalg.norm(nv),nv)
                nro = r_ao / np.linalg.norm(r_ao)
                
                
                # SPEED
                
                #print('nro',np.linalg.norm(nro),nro)
                a = np.cross(nro.flatten(),nv.flatten())
                #print('a',np.linalg.norm(a),a)
                b = np.cross(a.flatten(),nro.flatten())
                b = b /np.linalg.norm(b)
                #print('b',np.linalg.norm(b),b)
                P_aux = np.zeros((3,1)) 
                P_aux[0,0] = -float(xn)
                P_aux[1,0] = -float(yn) 
                ang_aux = np.arccos(np.matmul(b,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b = b
                else :
                    b = -b
  
                
                # AXIS Z
                
                a1 = np.cross(nro.flatten(),P_aux.flatten())
                #print('a',np.linalg.norm(a1))
                b1 = np.cross(a1.flatten(),nro.flatten())
                #print('b',np.linalg.norm(b1))
                b1 = b1 /np.linalg.norm(b1)
                ang_aux = np.arccos(np.matmul(b1,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b1)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b1 = b1
                else :
                    b1 = -b1
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Fd = np.matmul(V.reshape([1,3]),ur.reshape([3,1]))
                Ft = kd1*b.reshape([3,1])*Fk + kd2*b1.reshape([3,1])*Fk - kd3*Fk*ur.reshape([3,1]) 
                #print('Ft',Ft)
                print('P',P)
                print('nro',nro)
                print('b',b)
                print('b1',b1)
                print('ur',ur)
                #print('hv',hv)
                vx1 = np.exp(-dmin*tau)*Ft[0,0] + (1-np.exp(-dmin*tau))*hv[0,0]
                vy1 = np.exp(-dmin*tau)*Ft[1,0] + (1-np.exp(-dmin*tau))*hv[1,0]
                vz1 = np.exp(-dmin*tau)*Ft[2,0] + (1-np.exp(-dmin*tau))*hv[2,0]
            else:                  
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Ft = - kd3*Fk*ur.reshape([3,1]) 
                vx1 = 0.8*np.exp(-dmin*tau)*Ft[0,0] + 1.2*(1-np.exp(-dmin*tau))*hv[0,0]
                vy1 = 0.8*np.exp(-dmin*tau)*Ft[1,0] + 1.2*(1-np.exp(-dmin*tau))*hv[1,0]
                vz1 = 0.8*np.exp(-dmin*tau)*Ft[2,0] + 1.2*(1-np.exp(-dmin*tau))*hv[2,0]
        else:
            vx1 = hv[0,0]
            vy1 = hv[1,0]
            vz1 = hv[2,0]
        
        
        
        VMAX = np.sqrt(vx1*vx1+vy1*vy1+vz1*vz1) 
        
        if (VMAX > vmax):
            vx1 = vx1*vmax/VMAX
            vy1 = vy1*vmax/VMAX
            vz1 = vz1*vmax/VMAX



        V[0,0] = vx1
        V[1,0] = vy1
        V[2,0] = vz1


        try:
            
            
            rob.speedl((vx1 , vy1, vz1 , 0, 0, 0), acc=0.3, min_time=3);
        except:
            print('sent')

        time.sleep(0)

        toc = time.clock()
        print(e)
        f.write(str(xn)+";"+str(yn)+";"+str(zn)+";"+str(tic)+";"+str(dmin)+";"+str(kd1)+";"+str(kd2)+";"+str(kd3)+";"+str(tau)+";"+str(vmax)+";"+str(kangle)+"\n")
            
else:
    print('finishing')
    try:
        rob.stopl()
    except:
        print('No')

In [ ]:
#avoidance controller 2 + trajectory points BT
import time
import sys
import urx
import numpy as np
import keyboard
import json
import numpy as np
from numpy.linalg import inv
import time

interval = True
f= open("paper8.txt","w+")

#v =triad_openvr.triad_openvr()

try:
    rob = urx.Robot("192.168.88.240")
except:
    print('robot error')
    interval = False


if interval:

    #Controller
    dt = 0.1


    print('1')
    P = np.array([0.2,0.2,0.2]).reshape(3,1);
    P_obs = np.array([0.2,0.2,0.2]).reshape(3,1);
    V = np.array([0.0,0.0,0.0]).reshape(3,1)
    Po = np.array([2.1,2.1,10.1]).reshape(3,1);
    e = np.zeros((3,1));
    ea = np.zeros((3,1));
    lamda = 0.01
    k2 = (np.diag([1.2,2.3,3.3]));
    k1 = ( np.diag([1.1,2.3,1.3]));
    k3 = ( np.diag([1.1,2.3,1.3]));
    with open('bt.json', 'r') as fa:
        data = json.load(fa)
        n_points = data['general']['n']
        n_counter = 1
        x_refr = data['position'+str(n_counter)]['xref']
        y_refr = data['position'+str(n_counter)]['yref']
        z_refr = data['position'+str(n_counter)]['zref']
    to = time.clock()
        
    while True:
        tic = time.clock()
        
        try:
            with open('data2.json', 'r') as fa:
                data = json.load(fa)
                k1[0,0] = data['controller']['k11']
                k1[1,1] = data['controller']['k12']
                k1[2,2] = data['controller']['k13']
                k2[0,0] = data['controller']['k21']
                k2[1,1] = data['controller']['k22']
                k2[2,2] = data['controller']['k23']
                k3[0,0] = data['controller']['k31']
                k3[1,1] = data['controller']['k32']
                k3[2,2] = data['controller']['k33']
                vmax = data['controller']['vmax']
                vmax2 = data['poss_obs']['vmax']
                T = data['poss']['T']
                xo_refr = data['poss_obs']['xref']
                yo_refr = data['poss_obs']['yref']
                zo_refr = data['poss_obs']['zref']
                dmin = data['poss_obs']['dmin']
                kd1 = data['poss_obs']['kd1']
                kd2 = data['poss_obs']['kd2']
                kd3 = data['poss_obs']['kd3']
                tau = data['poss_obs']['tau']
                kangle = data['poss_obs']['kangle']
        except:
            print('file error')


        t = rob.get_pose()
      #  print("Transformation from base to tcp is: ", t)

        
        pose = rob.getl()
        xn = float(pose[0])
        yn = float(pose[1])
        zn = float(pose[2])
        
        P = np.zeros((3,1));
        P[0,0] = float(xn)
        P[1,0] = float(yn)
        P[2,0] = float(zn)
    #    print('Actual Point',P)
        
        if np.linalg.norm(e) < 0.01 :
            n_counter = n_counter + 1
            if n_counter > n_points :
                n_counter = 1
                
            with open('bt.json', 'r') as fa:
                data = json.load(fa)
                n_points = data['general']['n']
                x_refr = data['position'+str(n_counter)]['xref']
                y_refr = data['position'+str(n_counter)]['yref']
                z_refr = data['position'+str(n_counter)]['zref']

        Pa = np.zeros((3,1));    
        Pa[0,0] = float(x_refr)
        Pa[1,0] = float(y_refr)
        Pa[2,0] = float(z_refr)
        print('Desired Point',Pa)
        
        P_obs = np.zeros((3,1));   
        P_obs[0,0] = float(xo_refr)
        P_obs[1,0] = float(yo_refr)
        P_obs[2,0] = float(zo_refr)
     #   print('Obstacle point',P_obs)
    
        e = Pa - P      
        
        hv = np.matmul(k2,np.tanh(np.matmul(k1,e) + np.matmul(k3,e-ea)))
        ea = e
        
        r_ao = P_obs - P
        n_r_ao = np.linalg.norm(r_ao)
        
        if n_r_ao < dmin:
            #First collision condition
            print('collision condition 1')
            teta = np.arccos(np.matmul(r_ao.transpose(),e)/ (np.linalg.norm(r_ao)*np.linalg.norm(e)) )
            alfa = np.arccos(np.matmul(r_ao.transpose(),-P)/ (np.linalg.norm(r_ao)*np.linalg.norm(P)) ) 
            beta = np.arccos(np.matmul(e.transpose(),-P)/ (np.linalg.norm(P)*np.linalg.norm(e)) )
            print('r_ao',r_ao)
            print('v',V)
            
            print('ang',alfa, beta+teta)
            if  teta < np.pi*kangle/180:
                #seocnd collision condition
                nr = P / np.linalg.norm(P)
                print('collision condition 2')
                nrg = e / np.linalg.norm(e)
                #print(np.linalg.norm(e))
                nv = V / np.linalg.norm(V)
                #print('nv',np.linalg.norm(nv),nv)
                nro = r_ao / np.linalg.norm(r_ao)
                
                
                # SPEED
                
                #print('nro',np.linalg.norm(nro),nro)
                a = np.cross(nro.flatten(),nv.flatten())
                #print('a',np.linalg.norm(a),a)
                b = np.cross(a.flatten(),nro.flatten())
                b = b /np.linalg.norm(b)
                #print('b',np.linalg.norm(b),b)
                P_aux = np.zeros((3,1)) 
                P_aux[0,0] = -float(xn)
                P_aux[1,0] = -float(yn) 
                ang_aux = np.arccos(np.matmul(b,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b = b
                else :
                    b = -b
  
                
                # AXIS Z
                
                a1 = np.cross(nro.flatten(),P_aux.flatten())
                #print('a',np.linalg.norm(a1))
                b1 = np.cross(a1.flatten(),nro.flatten())
                #print('b',np.linalg.norm(b1))
                b1 = b1 /np.linalg.norm(b1)
                ang_aux = np.arccos(np.matmul(b1,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b1)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b1 = b1
                else :
                    b1 = -b1
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Fd = np.matmul(V.reshape([1,3]),ur.reshape([3,1]))
                Ft = kd1*b.reshape([3,1])*Fk + kd2*b1.reshape([3,1])*Fk - kd3*Fk*ur.reshape([3,1]) 
                #print('Ft',Ft)
                print('P',P)
                print('nro',nro)
                print('b',b)
                print('b1',b1)
                print('ur',ur)
                #print('hv',hv)
                vx1 = np.exp(-dmin*tau)*Ft[0,0] + (1-np.exp(-dmin*tau))*hv[0,0]
                vy1 = np.exp(-dmin*tau)*Ft[1,0] + (1-np.exp(-dmin*tau))*hv[1,0]
                vz1 = np.exp(-dmin*tau)*Ft[2,0] + (1-np.exp(-dmin*tau))*hv[2,0]
            else:                  
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Ft = - kd3*Fk*ur.reshape([3,1]) 
                vx1 = 0.8*np.exp(-dmin*tau)*Ft[0,0] + 1.2*(1-np.exp(-dmin*tau))*hv[0,0]
                vy1 = 0.8*np.exp(-dmin*tau)*Ft[1,0] + 1.2*(1-np.exp(-dmin*tau))*hv[1,0]
                vz1 = 0.8*np.exp(-dmin*tau)*Ft[2,0] + 1.2*(1-np.exp(-dmin*tau))*hv[2,0]
        else:
            vx1 = hv[0,0]
            vy1 = hv[1,0]
            vz1 = hv[2,0]
        
        
        
        VMAX = np.sqrt(vx1*vx1+vy1*vy1+vz1*vz1) 
        
        if (VMAX > vmax):
            vx1 = vx1*vmax/VMAX
            vy1 = vy1*vmax/VMAX
            vz1 = vz1*vmax/VMAX



        V[0,0] = vx1
        V[1,0] = vy1
        V[2,0] = vz1


        try:
            
            
            rob.speedl((vx1 , vy1, vz1 , 0, 0, 0), acc=0.3, min_time=1);
        except:
            print('sent')

        time.sleep(0)

        toc = time.clock()
        print(e)
        f.write(str(xn)+";"+str(yn)+";"+str(zn)+";"+str(tic)+";"+str(dmin)+";"+str(kd1)+";"+str(kd2)+";"+str(kd3)+";"+str(tau)+";"+str(vmax)+";"+str(kangle)+"\n")
            
else:
    print('finishing')
    try:
        rob.stopl()
    except:
        print('No')

In [ ]:
#avoidance controller 2 + trajectory points BT + tracker
import time
import sys
import urx
import numpy as np
import keyboard
import json
import numpy as np
from numpy.linalg import inv
import time
import triad_openvr

interval = True
f= open("paper8.txt","w+")

#calibration values
xo = 0.012 -0.03
yo = 0.01 + 0.056
zo = 0.211 + 0.118 - 0.345
x = 0.03977084159851074  
y = 0.8335312008857727
z = 0.9234819412231445  

#v =triad_openvr.triad_openvr()

try:
    rob = urx.Robot("192.168.88.240")
except:
    print('robot error')
    interval = False

v = triad_openvr.triad_openvr()
if interval:

    #Controller
    dt = 0.1


    print('1')
    P = np.array([0.2,0.2,0.2]).reshape(3,1);
    P_obs = np.array([0.2,0.2,0.2]).reshape(3,1);
    V = np.array([0.0,0.0,0.0]).reshape(3,1)
    Po = np.array([2.1,2.1,10.1]).reshape(3,1);
    e = np.zeros((3,1));
    ea = np.zeros((3,1));
    lamda = 0.01
    k2 = (np.diag([1.2,2.3,3.3]));
    k1 = ( np.diag([1.1,2.3,1.3]));
    k3 = ( np.diag([1.1,2.3,1.3]));
    with open('bt.json', 'r') as fa:
        data = json.load(fa)
        n_points = data['general']['n']
        n_counter = 1
        x_refr = data['position'+str(n_counter)]['xref']
        y_refr = data['position'+str(n_counter)]['yref']
        z_refr = data['position'+str(n_counter)]['zref']
    to = time.clock()
        
    while True:
        tic = time.clock()
        
        try:
            with open('data2.json', 'r') as fa:
                data = json.load(fa)
                k1[0,0] = data['controller']['k11']
                k1[1,1] = data['controller']['k12']
                k1[2,2] = data['controller']['k13']
                k2[0,0] = data['controller']['k21']
                k2[1,1] = data['controller']['k22']
                k2[2,2] = data['controller']['k23']
                k3[0,0] = data['controller']['k31']
                k3[1,1] = data['controller']['k32']
                k3[2,2] = data['controller']['k33']
                vmax = data['controller']['vmax']
                vmax2 = data['poss_obs']['vmax']
                T = data['poss']['T']
                xo_refr = data['poss_obs']['xref']
                yo_refr = data['poss_obs']['yref']
                zo_refr = data['poss_obs']['zref']
                dmin = data['poss_obs']['dmin']
                kd1 = data['poss_obs']['kd1']
                kd2 = data['poss_obs']['kd2']
                kd3 = data['poss_obs']['kd3']
                tau = data['poss_obs']['tau']
                kangle = data['poss_obs']['kangle']
        except:
            print('file error')


        #try:
        [x1,y1,z1,yaw1,pitch1,roll1] = v.devices["tracker_1"].get_pose_euler();
        pos_x = x1 - x 
        pos_y = y1 - y 
        pos_z = z1 - z 

        print('x:', pos_x,'y:', pos_y,'z:', pos_z)
        xo_refr = -pos_z + xo
        yo_refr = -pos_x + yo
        zo_refr = pos_y + zo 
        print('x_robot:', xo_refr,'y_robot:', yo_refr,'z_robot:', zo_refr)
        #except:
        #    xo_refr = 0
        #    yo_refr = 0
        #    zo_refr = 0 

        
        t = rob.get_pose()
      #  print("Transformation from base to tcp is: ", t)

        
        pose = rob.getl()
        xn = float(pose[0])
        yn = float(pose[1])
        zn = float(pose[2])
        
        P = np.zeros((3,1));
        P[0,0] = float(xn)
        P[1,0] = float(yn)
        P[2,0] = float(zn)
    #    print('Actual Point',P)
        
        if np.linalg.norm(e) < 0.01 :
            n_counter = n_counter + 1
            if n_counter > n_points :
                n_counter = 1
                
            with open('bt.json', 'r') as fa:
                data = json.load(fa)
                n_points = data['general']['n']
                x_refr = data['position'+str(n_counter)]['xref']
                y_refr = data['position'+str(n_counter)]['yref']
                z_refr = data['position'+str(n_counter)]['zref']

        Pa = np.zeros((3,1));    
        Pa[0,0] = float(x_refr)
        Pa[1,0] = float(y_refr)
        Pa[2,0] = float(z_refr)
        print('Desired Point',Pa)
        
        P_obs = np.zeros((3,1));   
        P_obs[0,0] = float(xo_refr)
        P_obs[1,0] = float(yo_refr)
        P_obs[2,0] = float(zo_refr)
     #   print('Obstacle point',P_obs)
    
        e = Pa - P      
        
        hv = np.matmul(k2,np.tanh(np.matmul(k1,e) + np.matmul(k3,e-ea)))
        ea = e
        
        r_ao = P_obs - P
        n_r_ao = np.linalg.norm(r_ao)
        
        if n_r_ao < dmin:
            #First collision condition
            print('collision condition 1')
            teta = np.arccos(np.matmul(r_ao.transpose(),e)/ (np.linalg.norm(r_ao)*np.linalg.norm(e)) )
            alfa = np.arccos(np.matmul(r_ao.transpose(),-P)/ (np.linalg.norm(r_ao)*np.linalg.norm(P)) ) 
            beta = np.arccos(np.matmul(e.transpose(),-P)/ (np.linalg.norm(P)*np.linalg.norm(e)) )
            print('r_ao',r_ao)
            print('v',V)
            
            print('ang',alfa, beta+teta)
            if  teta < np.pi*kangle/180:
                #seocnd collision condition
                nr = P / np.linalg.norm(P)
                print('collision condition 2')
                nrg = e / np.linalg.norm(e)
                #print(np.linalg.norm(e))
                nv = V / np.linalg.norm(V)
                #print('nv',np.linalg.norm(nv),nv)
                nro = r_ao / np.linalg.norm(r_ao)
                
                
                # SPEED
                
                #print('nro',np.linalg.norm(nro),nro)
                a = np.cross(nro.flatten(),nv.flatten())
                #print('a',np.linalg.norm(a),a)
                b = np.cross(a.flatten(),nro.flatten())
                b = b /np.linalg.norm(b)
                #print('b',np.linalg.norm(b),b)
                P_aux = np.zeros((3,1)) 
                P_aux[0,0] = -float(xn)
                P_aux[1,0] = -float(yn) 
                ang_aux = np.arccos(np.matmul(b,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b = b
                else :
                    b = -b
  
                
                # AXIS Z
                
                a1 = np.cross(nro.flatten(),P_aux.flatten())
                #print('a',np.linalg.norm(a1))
                b1 = np.cross(a1.flatten(),nro.flatten())
                #print('b',np.linalg.norm(b1))
                b1 = b1 /np.linalg.norm(b1)
                ang_aux = np.arccos(np.matmul(b1,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b1)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b1 = b1
                else :
                    b1 = -b1
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Fd = np.matmul(V.reshape([1,3]),ur.reshape([3,1]))
                Ft = kd1*b.reshape([3,1])*Fk + kd2*b1.reshape([3,1])*Fk - kd3*Fk*ur.reshape([3,1]) 
                #print('Ft',Ft)
                print('P',P)
                print('nro',nro)
                print('b',b)
                print('b1',b1)
                print('ur',ur)
                #print('hv',hv)
                vx1 = np.exp(-dmin*tau)*Ft[0,0] + (1-np.exp(-dmin*tau))*hv[0,0]
                vy1 = np.exp(-dmin*tau)*Ft[1,0] + (1-np.exp(-dmin*tau))*hv[1,0]
                vz1 = np.exp(-dmin*tau)*Ft[2,0] + (1-np.exp(-dmin*tau))*hv[2,0]
            else:                  
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Ft = - kd3*Fk*ur.reshape([3,1]) 
                vx1 = 0.8*np.exp(-dmin*tau)*Ft[0,0] + 1.2*(1-np.exp(-dmin*tau))*hv[0,0]
                vy1 = 0.8*np.exp(-dmin*tau)*Ft[1,0] + 1.2*(1-np.exp(-dmin*tau))*hv[1,0]
                vz1 = 0.8*np.exp(-dmin*tau)*Ft[2,0] + 1.2*(1-np.exp(-dmin*tau))*hv[2,0]
        else:
            vx1 = hv[0,0]
            vy1 = hv[1,0]
            vz1 = hv[2,0]
        
        
        
        VMAX = np.sqrt(vx1*vx1+vy1*vy1+vz1*vz1) 
        
        if (VMAX > vmax):
            vx1 = vx1*vmax/VMAX
            vy1 = vy1*vmax/VMAX
            vz1 = vz1*vmax/VMAX



        V[0,0] = vx1
        V[1,0] = vy1
        V[2,0] = vz1


        try:
            
            
            rob.speedl((vx1 , vy1, vz1 , 0, 0, 0), acc=0.3, min_time=1);
        except:
            print('sent')

        time.sleep(0)

        toc = time.clock()
        print(e)
        f.write(str(xn)+";"+str(yn)+";"+str(zn)+";"+str(xo_refr)+";"+str(yo_refr)+";"+str(zo_refr)+";"+str(tic)+";"+str(dmin)+";"+str(kd1)+";"+str(kd2)+";"+str(kd3)+";"+str(tau)+";"+str(vmax)+";"+str(kangle)+"\n")
            
else:
    print('finishing')
    try:
        rob.stopl()
    except:
        print('No')

In [ ]:
#avoidance controller 2 + trajectory points BT + tracker + selfdriving
import time
import sys
import urx
import numpy as np
import keyboard
import json
import numpy as np
from numpy.linalg import inv
import time
import triad_openvr

interval = True
f= open("paper8.txt","w+")

#calibration values
xo = 0.012 -0.03
yo = 0.01 + 0.056
zo = 0.211 + 0.118 - 0.345
x = 0.03977084159851074  
y = 0.8335312008857727
z = 0.9234819412231445  

#v =triad_openvr.triad_openvr()

try:
    rob = urx.Robot("192.168.88.240")
except:
    print('robot error')
    interval = False

v = triad_openvr.triad_openvr()
if interval:

    #Controller
    dt = 0.1


    print('1')
    P = np.array([0.2,0.2,0.2]).reshape(3,1);
    P_obs = np.array([0.2,0.2,0.2]).reshape(3,1);
    V = np.array([0.0,0.0,0.0]).reshape(3,1)
    Po = np.array([2.1,2.1,10.1]).reshape(3,1);
    e = np.zeros((3,1));
    ea = np.zeros((3,1));
    lamda = 0.01
    k2 = (np.diag([1.2,2.3,3.3]));
    k1 = ( np.diag([1.1,2.3,1.3]));
    k3 = ( np.diag([1.1,2.3,1.3]));
    with open('bt.json', 'r') as fa:
        data = json.load(fa)
        n_points = data['general']['n']
        n_counter = 1
        x_refr = data['position'+str(n_counter)]['xref']
        y_refr = data['position'+str(n_counter)]['yref']
        z_refr = data['position'+str(n_counter)]['zref']
    to = time.clock()
        
    while True:
        tic = time.clock()
        
        try:
            with open('data2.json', 'r') as fa:
                data = json.load(fa)
                k1[0,0] = data['controller']['k11']
                k1[1,1] = data['controller']['k12']
                k1[2,2] = data['controller']['k13']
                k2[0,0] = data['controller']['k21']
                k2[1,1] = data['controller']['k22']
                k2[2,2] = data['controller']['k23']
                k3[0,0] = data['controller']['k31']
                k3[1,1] = data['controller']['k32']
                k3[2,2] = data['controller']['k33']
                vmax = data['controller']['vmax']
                vmax2 = data['poss_obs']['vmax']
                T = data['poss']['T']
                xo_refr = data['poss_obs']['xref']
                yo_refr = data['poss_obs']['yref']
                zo_refr = data['poss_obs']['zref']
                dmin = data['poss_obs']['dmin']
                dmin2 = data['poss_obs']['dmin2']
                kd1 = data['poss_obs']['kd1']
                kd2 = data['poss_obs']['kd2']
                kd3 = data['poss_obs']['kd3']
                tau = data['poss_obs']['tau']
                kangle = data['poss_obs']['kangle']
        except:
            print('file error')


        try:
            [x1,y1,z1,yaw1,pitch1,roll1] = v.devices["tracker_1"].get_pose_euler();
            pos_x = x1 - x 
            pos_y = y1 - y 
            pos_z = z1 - z 

            print('x:', pos_x,'y:', pos_y,'z:', pos_z)
            xo_refr = -pos_z + xo
            yo_refr = -pos_x + yo
            zo_refr = pos_y + zo 
            print('x_robot:', xo_refr,'y_robot:', yo_refr,'z_robot:', zo_refr)
        except:
            xo_refr = 0
            yo_refr = 0
            zo_refr = 0 

        
        t = rob.get_pose()
      #  print("Transformation from base to tcp is: ", t)

        
        pose = rob.getl()
        xn = float(pose[0])
        yn = float(pose[1])
        zn = float(pose[2])
        
        P = np.zeros((3,1));
        P[0,0] = float(xn)
        P[1,0] = float(yn)
        P[2,0] = float(zn)
    #    print('Actual Point',P)
        
        if np.linalg.norm(e) < 0.01 :
            n_counter = n_counter + 1
            if n_counter > n_points :
                n_counter = 1
                
            with open('bt.json', 'r') as fa:
                data = json.load(fa)
                n_points = data['general']['n']
                x_refr = data['position'+str(n_counter)]['xref']
                y_refr = data['position'+str(n_counter)]['yref']
                z_refr = data['position'+str(n_counter)]['zref']

        Pa = np.zeros((3,1));    
        Pa[0,0] = float(x_refr)
        Pa[1,0] = float(y_refr)
        Pa[2,0] = float(z_refr)
        print('Desired Point',Pa)
        
        P_obs = np.zeros((3,1));   
        P_obs[0,0] = float(xo_refr)
        P_obs[1,0] = float(yo_refr)
        P_obs[2,0] = float(zo_refr)
     #   print('Obstacle point',P_obs)
    
        e = Pa - P      
        
        hv = np.matmul(k2,np.tanh(np.matmul(k1,e) + np.matmul(k3,e-ea)))
        ea = e
        
        r_ao = P_obs - P
        n_r_ao = np.linalg.norm(r_ao)
        
        if n_r_ao < dmin and n_r_ao > dmin2:
            #First collision condition
            print('collision condition 1')
            teta = np.arccos(np.matmul(r_ao.transpose(),e)/ (np.linalg.norm(r_ao)*np.linalg.norm(e)) )
            alfa = np.arccos(np.matmul(r_ao.transpose(),-P)/ (np.linalg.norm(r_ao)*np.linalg.norm(P)) ) 
            beta = np.arccos(np.matmul(e.transpose(),-P)/ (np.linalg.norm(P)*np.linalg.norm(e)) )
            print('r_ao',r_ao)
            print('v',V)
            
            print('ang',alfa, beta+teta)
            if  teta < np.pi*kangle/180:
                #seocnd collision condition
                nr = P / np.linalg.norm(P)
                print('collision condition 2')
                nrg = e / np.linalg.norm(e)
                #print(np.linalg.norm(e))
                nv = V / np.linalg.norm(V)
                #print('nv',np.linalg.norm(nv),nv)
                nro = r_ao / np.linalg.norm(r_ao)
                
                
                # SPEED
                
                #print('nro',np.linalg.norm(nro),nro)
                a = np.cross(nro.flatten(),nv.flatten())
                #print('a',np.linalg.norm(a),a)
                b = np.cross(a.flatten(),nro.flatten())
                b = b /np.linalg.norm(b)
                #print('b',np.linalg.norm(b),b)
                P_aux = np.zeros((3,1)) 
                P_aux[0,0] = -float(xn)
                P_aux[1,0] = -float(yn) 
                ang_aux = np.arccos(np.matmul(b,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b = b
                else :
                    b = -b
  
                
                # AXIS Z
                
                a1 = np.cross(nro.flatten(),P_aux.flatten())
                #print('a',np.linalg.norm(a1))
                b1 = np.cross(a1.flatten(),nro.flatten())
                #print('b',np.linalg.norm(b1))
                b1 = b1 /np.linalg.norm(b1)
                ang_aux = np.arccos(np.matmul(b1,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b1)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b1 = b1
                else :
                    b1 = -b1
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Fd = np.matmul(V.reshape([1,3]),ur.reshape([3,1]))
                Ft = kd1*b.reshape([3,1])*Fk + kd2*b1.reshape([3,1])*Fk - kd3*Fk*ur.reshape([3,1]) 
                #print('Ft',Ft)
                print('P',P)
                print('nro',nro)
                print('b',b)
                print('b1',b1)
                print('ur',ur)
                #print('hv',hv)
                vx1 = np.exp(-dmin*tau)*Ft[0,0] + (1-np.exp(-dmin*tau))*hv[0,0]
                vy1 = np.exp(-dmin*tau)*Ft[1,0] + (1-np.exp(-dmin*tau))*hv[1,0]
                vz1 = np.exp(-dmin*tau)*Ft[2,0] + (1-np.exp(-dmin*tau))*hv[2,0]
            else:                  
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Ft = - kd3*Fk*ur.reshape([3,1]) 
                vx1 = 0.8*np.exp(-dmin*tau)*hv[0,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[0,0]
                vy1 = 0.8*np.exp(-dmin*tau)*hv[1,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[1,0]
                vz1 = 0.8*np.exp(-dmin*tau)*hv[2,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[2,0]
        elif n_r_ao < dmin2:
            vx1 = 0
            vy1 = 0
            vz1 = 0
            rob.set_freedrive(1)
            
            while n_r_ao < dmin2+0.1:
                try:
                    [x1,y1,z1,yaw1,pitch1,roll1] = v.devices["tracker_1"].get_pose_euler();
                    pos_x = x1 - x 
                    pos_y = y1 - y 
                    pos_z = z1 - z 

                    print('x:', pos_x,'y:', pos_y,'z:', pos_z)
                    xo_refr = -pos_z + xo
                    yo_refr = -pos_x + yo
                    zo_refr = pos_y + zo 
                    print('x_robot:', xo_refr,'y_robot:', yo_refr,'z_robot:', zo_refr)
                except:
                    xo_refr = xo_refr
                    yo_refr = yo_refr
                    zo_refr = zo_refr 


                t = rob.get_pose()
              #  print("Transformation from base to tcp is: ", t)


                pose = rob.getl()
                xn = float(pose[0])
                yn = float(pose[1])
                zn = float(pose[2])

                P = np.zeros((3,1));
                P[0,0] = float(xn)
                P[1,0] = float(yn)
                P[2,0] = float(zn)



                P_obs = np.zeros((3,1));   
                P_obs[0,0] = float(xo_refr)
                P_obs[1,0] = float(yo_refr)
                P_obs[2,0] = float(zo_refr)

                r_ao = P_obs - P
                n_r_ao = np.linalg.norm(r_ao)
                rob.set_freedrive(1)
                
            rob.set_freedrive(0)
        
        
        else :
            vx1 = hv[0,0]
            vy1 = hv[1,0]
            vz1 = hv[2,0]
        
        
        
        VMAX = np.sqrt(vx1*vx1+vy1*vy1+vz1*vz1) 
        
        if (VMAX > vmax):
            vx1 = vx1*vmax/VMAX
            vy1 = vy1*vmax/VMAX
            vz1 = vz1*vmax/VMAX



        V[0,0] = vx1
        V[1,0] = vy1
        V[2,0] = vz1


        try:
            
            
            rob.speedl((vx1 , vy1, vz1 , 0, 0, 0), acc=0.3, min_time=1);
        except:
            print('sent')

        time.sleep(0)

        toc = time.clock()
        print(e)
        f.write(str(xn)+";"+str(yn)+";"+str(zn)+";"+str(xo_refr)+";"+str(yo_refr)+";"+str(zo_refr)+";"+str(tic)+";"+str(dmin)+";"+str(kd1)+";"+str(kd2)+";"+str(kd3)+";"+str(tau)+";"+str(vmax)+";"+str(kangle)+"\n")
            
else:
    print('finishing')
    try:
        rob.stopl()
    except:
        print('No')

In [ ]:
#avoidance controller 2 + trajectory + tracker
import time
import sys
import urx
import numpy as np
import keyboard
import json
import numpy as np
from numpy.linalg import inv
import time

interval = True
f= open("paper8.txt","w+")

#v =triad_openvr.triad_openvr()

try:
    rob = urx.Robot("192.168.88.240")
except:
    print('robot error')
    interval = False

#calibration values
xo = 0.012 -0.03
yo = 0.01 + 0.056
zo = 0.211 + 0.118 - 0.345
x = 0.03977084159851074  
y = 0.8335312008857727
z = 0.9234819412231445  

if interval:

    #Controller
    dt = 0.1


    print('1')
    P = np.array([0.2,0.2,0.2]).reshape(3,1);
    P_obs = np.array([0.2,0.2,0.2]).reshape(3,1);
    V = np.array([0.0,0.0,0.0]).reshape(3,1)
    Po = np.array([2.1,2.1,10.1]).reshape(3,1);
    e = np.zeros((3,1));
    ea = np.zeros((3,1));
    lamda = 0.01
    k2 = (np.diag([1.2,2.3,3.3]));
    k1 = ( np.diag([1.1,2.3,1.3]));
    k3 = ( np.diag([1.1,2.3,1.3]));

    to = time.clock()
        
    while True:
        tic = time.clock()
        
        try:
            with open('data2.json', 'r') as fa:
                data = json.load(fa)
                k1[0,0] = data['controller']['k11']
                k1[1,1] = data['controller']['k12']
                k1[2,2] = data['controller']['k13']
                k2[0,0] = data['controller']['k21']
                k2[1,1] = data['controller']['k22']
                k2[2,2] = data['controller']['k23']
                k3[0,0] = data['controller']['k31']
                k3[1,1] = data['controller']['k32']
                k3[2,2] = data['controller']['k33']
                vmax = data['controller']['vmax']
                vmax2 = data['poss_obs']['vmax']
                x_refr = data['poss']['xref']
                y_refr = data['poss']['yref']
                z_refr = data['poss']['zref']
                T = data['poss']['T']
                xo_refr = data['poss_obs']['xref']
                yo_refr = data['poss_obs']['yref']
                zo_refr = data['poss_obs']['zref']
                dmin = data['poss_obs']['dmin']
                kd1 = data['poss_obs']['kd1']
                kd2 = data['poss_obs']['kd2']
                kd3 = data['poss_obs']['kd3']
                tau = data['poss_obs']['tau']
                kangle = data['poss_obs']['kangle']
        except:
            print('file error')


        t = rob.get_pose()
      #  print("Transformation from base to tcp is: ", t)
    
        #try:
        [x1,y1,z1,yaw1,pitch1,roll1] = v.devices["tracker_1"].get_pose_euler();
        pos_x = x1 - x 
        pos_y = y1 - y 
        pos_z = z1 - z 

        print('x:', pos_x,'y:', pos_y,'z:', pos_z)
        xo_refr = -pos_z + xo
        yo_refr = -pos_x + yo
        zo_refr = pos_y + zo 
        print('x_robot:', xo_refr,'y_robot:', yo_refr,'z_robot:', zo_refr)
        #except:
        #    xo_refr = 0
        #    yo_refr = 0
        #    zo_refr = 0 

        t = rob.get_pose()
      #  print("Transformation from base to tcp is: ", t)
    
        #try:
        [x1,y1,z1,yaw1,pitch1,roll1] = v.devices["tracker_1"].get_pose_euler();
        pos_x = x1 - x 
        pos_y = y1 - y 
        pos_z = z1 - z 

        print('x:', pos_x,'y:', pos_y,'z:', pos_z)
        xo_refr = -pos_z + xo
        yo_refr = -pos_x + yo
        zo_refr = pos_y + zo 
        print('x_robot:', xo_refr,'y_robot:', yo_refr,'z_robot:', zo_refr)
        #except:
        #    xo_refr = 0
        #    yo_refr = 0
        #    zo_refr = 0 

        t = rob.get_pose()
      #  print("Transformation from base to tcp is: ", t)
        pose = rob.getl()
        xn = float(pose[0])
        yn = float(pose[1])
        zn = float(pose[2])
        
        P = np.zeros((3,1));
        P[0,0] = float(xn)
        P[1,0] = float(yn)
        P[2,0] = float(zn)
    #    print('Actual Point',P)
        
        A = 0.3
        Pa = np.zeros((3,1));   
        tact = tic - to;
        y_refr = (A * np.cos(2*np.pi*(tact)/T))
        z_refr = (A * np.sin(2*np.pi*(tact)/T) + 0.5)
        Pa[0,0] = float(x_refr)
        Pa[1,0] = float(y_refr)
        Pa[2,0] = float(z_refr)
        print('Desired Point',Pa)
        
        P_obs = np.zeros((3,1));   
        P_obs[0,0] = float(xo_refr)
        P_obs[1,0] = float(yo_refr)
        P_obs[2,0] = float(zo_refr)
     #   print('Obstacle point',P_obs)
    
        e = Pa - P      
        
        hv = np.matmul(k2,np.tanh(np.matmul(k1,e) + np.matmul(k3,e-ea)))
        ea = e
        
        r_ao = P_obs - P
        n_r_ao = np.linalg.norm(r_ao)
        
        if n_r_ao < dmin:
            #First collision condition
            print('collision condition 1')
            teta = np.arccos(np.matmul(r_ao.transpose(),e)/ (np.linalg.norm(r_ao)*np.linalg.norm(e)) )
            alfa = np.arccos(np.matmul(r_ao.transpose(),-P)/ (np.linalg.norm(r_ao)*np.linalg.norm(P)) ) 
            beta = np.arccos(np.matmul(e.transpose(),-P)/ (np.linalg.norm(P)*np.linalg.norm(e)) )
            print('r_ao',r_ao)
            print('v',V)
            
            print('ang',alfa, beta+teta)
            if  teta < np.pi*kangle/180:
                #seocnd collision condition
                nr = P / np.linalg.norm(P)
                print('collision condition 2')
                nrg = e / np.linalg.norm(e)
                #print(np.linalg.norm(e))
                nv = V / np.linalg.norm(V)
                #print('nv',np.linalg.norm(nv),nv)
                nro = r_ao / np.linalg.norm(r_ao)
                
                
                # SPEED
                
                #print('nro',np.linalg.norm(nro),nro)
                a = np.cross(nro.flatten(),nv.flatten())
                #print('a',np.linalg.norm(a),a)
                b = np.cross(a.flatten(),nro.flatten())
                b = b /np.linalg.norm(b)
                #print('b',np.linalg.norm(b),b)
                P_aux = np.zeros((3,1)) 
                P_aux[0,0] = -float(xn)
                P_aux[1,0] = -float(yn) 
                ang_aux = np.arccos(np.matmul(b,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b = b
                else :
                    b = -b
  
                
                # AXIS Z
                
                a1 = np.cross(nro.flatten(),P_aux.flatten())
                #print('a',np.linalg.norm(a1))
                b1 = np.cross(a1.flatten(),nro.flatten())
                #print('b',np.linalg.norm(b1))
                b1 = b1 /np.linalg.norm(b1)
                ang_aux = np.arccos(np.matmul(b1,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b1)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b1 = b1
                else :
                    b1 = -b1
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Fd = np.matmul(V.reshape([1,3]),ur.reshape([3,1]))
                Ft = kd1*b.reshape([3,1])*Fk + kd2*b1.reshape([3,1])*Fk - kd3*Fk*ur.reshape([3,1]) 
                #print('Ft',Ft)
                print('P',P)
                print('nro',nro)
                print('b',b)
                print('b1',b1)
                print('ur',ur)
                #print('hv',hv)
                vx1 = np.exp(-dmin*tau)*Ft[0,0] + (1-np.exp(-dmin*tau))*hv[0,0]
                vy1 = np.exp(-dmin*tau)*Ft[1,0] + (1-np.exp(-dmin*tau))*hv[1,0]
                vz1 = np.exp(-dmin*tau)*Ft[2,0] + (1-np.exp(-dmin*tau))*hv[2,0]
            else:                  
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Ft = - kd3*Fk*ur.reshape([3,1]) 
                vx1 = 0.95*np.exp(-dmin*tau)*Ft[0,0] + 1.05*(1-np.exp(-dmin*tau))*hv[0,0]
                vy1 = 0.95*np.exp(-dmin*tau)*Ft[1,0] + 1.05*(1-np.exp(-dmin*tau))*hv[1,0]
                vz1 = 0.95*np.exp(-dmin*tau)*Ft[2,0] + 1.05*(1-np.exp(-dmin*tau))*hv[2,0]
        else:
            vx1 = hv[0,0]
            vy1 = hv[1,0]
            vz1 = hv[2,0]
        
        
        
        VMAX = np.sqrt(vx1*vx1+vy1*vy1+vz1*vz1) 
        
        if (VMAX > vmax):
            vx1 = vx1*vmax/VMAX
            vy1 = vy1*vmax/VMAX
            vz1 = vz1*vmax/VMAX



        V[0,0] = vx1
        V[1,0] = vy1
        V[2,0] = vz1


        try:
            
            
            rob.speedl((vx1 , vy1, vz1 , 0, 0, 0), acc=0.3, min_time=3);
        except:
            print('sent')

        time.sleep(0)

        toc = time.clock()
        print(e)
        f.write(str(xn)+";"+str(yn)+";"+str(zn)+";"+str(tic)+";"+str(dmin)+";"+str(kd1)+";"+str(kd2)+";"+str(kd3)+";"+str(tau)+";"+str(vmax)+";"+str(kangle)+"\n")
            
else:
    print('finishing')
    try:
        rob.stopl()
    except:
        print('No')

In [ ]:
np.matmul(r_ao.transpose(),r_ao)

In [ ]:
import triad_openvr
import time
import sys

v = triad_openvr.triad_openvr()
v.print_discovered_objects()

if len(sys.argv) == 1:
    interval = 1/250
elif len(sys.argv) == 2:
    interval = 1/float(sys.argv[1])
else:
    print("Invalid number of arguments")
    interval = False
    
if interval:
    while(True):
        start = time.time()
        txt = ""
        for each in v.devices["tracker_1"].get_pose_euler():
            txt += "%.4f" % each
            txt += " "
        print("\r" + txt, end="")
        sleep_time = interval-(time.time()-start)
        if sleep_time>0:
            time.sleep(sleep_time)

In [ ]:
import triad_openvr
import time
import sys
import urx
#calibration values
xo = 0.012 -0.03
yo = 0.01 + 0.056
zo = 0.211 + 0.118 - 0.345
x = 0.03977084159851074  
y = 0.8335312008857727
z = 0.9234819412231445  


rob = urx.Robot("192.168.88.240")
v = triad_openvr.triad_openvr()

In [ ]:
for i in range(0,1):
    start = time.time()

    [x1,y1,z1,yaw1,pitch1,roll1] = v.devices["tracker_1"].get_pose_euler();
    pos_x = x1 - x 
    pos_y = y1 - y 
    pos_z = z1 - z 
    print('x:',x1,'y:',y1,'z:',z1)
    print('x:', pos_x,'y:', pos_y,'z:', pos_z)
    x_refr = -pos_z + xo
    y_refr = -pos_x + yo
    z_refr = pos_y + zo 
    print('x_robot:', x_refr,'y_robot:', y_refr,'z_robot:', z_refr)
    t = rob.get_pose()
    print("Transformation from base to tcp is: ", t)
    rob.close()


In [ ]:
while True:
    [x1,y1,z1,yaw1,pitch1,roll1] = v.devices["tracker_1"].get_pose_euler();
    print(x1,y1,z1)
    

In [ ]:
[x,y,z,yaw,pitch,roll] = v.devices["tracker_1"].get_pose_euler()

In [ ]:
print(x,y,z)

In [ ]:
import time
import sys
import urx
import numpy as np
import keyboard
import json
import numpy as np
from numpy.linalg import inv
import triad_openvr
import sys


interval = True

xo = 0.012 -0.03
yo = 0.01 + 0.056
zo = 0.211 + 0.118
x = -0.2355482578277588  
y = 1.0630919933319092 
z = 0.7969908714294434 

v = triad_openvr.triad_openvr()

try:
    rob = urx.Robot("192.168.88.249")
except:
    print('robot error')
    interval = False


if interval:

    #Controller
    dt = 0.1


    print('1')
    P = np.array([0.2,0.2,0.2]).reshape(3,1);
    P_obs = np.array([0.2,0.2,0.2]).reshape(3,1);
    V = np.array([0.0,0.0,0.0]).reshape(3,1)
    Po = np.array([2.1,2.1,10.1]).reshape(3,1);
    e = np.zeros((3,1));
    ea = np.zeros((3,1));
    lamda = 0.01
    k2 = (np.diag([1.2,2.3,3.3]));
    k1 = ( np.diag([1.1,2.3,1.3]));
    k3 = ( np.diag([1.1,2.3,1.3]));

        
    while True:
        tic = time.clock()
        
        try:
            with open('data2.json', 'r') as fa:
                data = json.load(fa)
                k1[0,0] = data['controller']['k11']
                k1[1,1] = data['controller']['k12']
                k1[2,2] = data['controller']['k13']
                k2[0,0] = data['controller']['k21']
                k2[1,1] = data['controller']['k22']
                k2[2,2] = data['controller']['k23']
                k3[0,0] = data['controller']['k31']
                k3[1,1] = data['controller']['k32']
                k3[2,2] = data['controller']['k33']
                vmax = data['controller']['vmax']
                x_refr = data['poss']['xref']
                y_refr = data['poss']['yref']
                z_refr = data['poss']['zref']
                dmin = data['poss_obs']['dmin']
                kd1 = data['poss_obs']['kd1']
                kd2 = data['poss_obs']['kd2']
                kd3 = data['poss_obs']['kd3']
                tau = data['poss_obs']['tau']
        except:
            print('file error')

        #try:
        [x1,y1,z1,yaw1,pitch1,roll1] = v.devices["tracker_1"].get_pose_euler();
        pos_x = x1 - x 
        pos_y = y1 - y 
        pos_z = z1 - z 

        print('x:', pos_x,'y:', pos_y,'z:', pos_z)
        xo_refr = -pos_z + xo
        yo_refr = -pos_x + yo
        zo_refr = pos_y + zo 
        print('x_robot:', xo_refr,'y_robot:', yo_refr,'z_robot:', zo_refr)
        #except:
        #    xo_refr = 0
        #    yo_refr = 0
        #    zo_refr = 0 

        t = rob.get_pose()
      #  print("Transformation from base to tcp is: ", t)


        pose = rob.getl()
        xn = float(pose[0])
        yn = float(pose[1])
        zn = float(pose[2])
        
        P = np.zeros((3,1));
        P[0,0] = float(xn)
        P[1,0] = float(yn)
        P[2,0] = float(zn)
    #    print('Actual Point',P)
        
        Pa = np.zeros((3,1));   
        Pa[0,0] = float(x_refr)
        Pa[1,0] = float(y_refr)
        Pa[2,0] = float(z_refr)
     #   print('Desired Point',Po)
        
        P_obs = np.zeros((3,1));   
        P_obs[0,0] = float(xo_refr)
        P_obs[1,0] = float(yo_refr)
        P_obs[2,0] = float(zo_refr)
        print('Obstacle point',P_obs)
    
        e = Pa - P      
        
        hv = np.matmul(k2,np.tanh(np.matmul(k1,e) + np.matmul(k3,e-ea)))
        ea = e
        
        r_ao = P_obs - P
        n_r_ao = np.linalg.norm(r_ao)
        
        if n_r_ao < dmin :
            #First collision condition
            print('collision condition 1')
            teta = np.arccos(np.matmul(r_ao.transpose(),e)/ (np.linalg.norm(r_ao)*np.linalg.norm(e)) )
            alfa = np.arccos(np.matmul(r_ao.transpose(),-P)/ (np.linalg.norm(r_ao)*np.linalg.norm(P)) ) 
            beta = np.arccos(np.matmul(e.transpose(),-P)/ (np.linalg.norm(P)*np.linalg.norm(e)) )
            print('r_ao',r_ao)
            print('v',V)
            
            print('ang',alfa, beta+teta)
            if  beta + teta < 1.1 * np.pi:
                #seocnd collision condition
                nr = P / np.linalg.norm(P)
                print('collision condition 2')
                nrg = e / np.linalg.norm(e)
                #print(np.linalg.norm(e))
                nv = V / np.linalg.norm(V)
                #print('nv',np.linalg.norm(nv),nv)
                nro = r_ao / np.linalg.norm(r_ao)
                
                
                # SPEED
                
                #print('nro',np.linalg.norm(nro),nro)
                a = np.cross(nro.flatten(),nv.flatten())
                #print('a',np.linalg.norm(a),a)
                b = np.cross(a.flatten(),nro.flatten())
                b = b /np.linalg.norm(b)
                #print('b',np.linalg.norm(b),b)
                P_aux = np.zeros((3,1)) 
                P_aux[0,0] = -float(xn)
                P_aux[1,0] = -float(yn) 
                ang_aux = np.arccos(np.matmul(b,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b = b
                else :
                    b = -b
  
                
                # AXIS Z
                
                a1 = np.cross(nro.flatten(),P_aux.flatten())
                #print('a',np.linalg.norm(a1))
                b1 = np.cross(a1.flatten(),nro.flatten())
                #print('b',np.linalg.norm(b1))
                b1 = b1 /np.linalg.norm(b1)
                ang_aux = np.arccos(np.matmul(b1,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b1)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b1 = b1
                else :
                    b1 = -b1
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Fd = np.matmul(V.reshape([1,3]),ur.reshape([3,1]))
                Ft = kd1*b.reshape([3,1])*Fk + kd2*b1.reshape([3,1])*Fk - kd3*Fk*ur.reshape([3,1]) 
                #print('Ft',Ft)
                print('P',P)
                print('nro',nro)
                print('b',b)
                print('b1',b1)
                print('ur',ur)
                #print('hv',hv)
                vx1 = np.exp(-dmin*tau)*Ft[0,0] + (1-np.exp(-dmin*tau))*hv[0,0]
                vy1 = np.exp(-dmin*tau)*Ft[1,0] + (1-np.exp(-dmin*tau))*hv[1,0]
                vz1 = np.exp(-dmin*tau)*Ft[2,0] + (1-np.exp(-dmin*tau))*hv[2,0]
            else:                  
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Ft = - kd3*Fk*ur.reshape([3,1]) 
                vx1 = np.exp(-dmin*tau)*Ft[0,0] + (1-np.exp(-dmin*tau))*hv[0,0]
                vy1 = np.exp(-dmin*tau)*Ft[1,0] + (1-np.exp(-dmin*tau))*hv[1,0]
                vz1 = np.exp(-dmin*tau)*Ft[2,0] + (1-np.exp(-dmin*tau))*hv[2,0]
                
        
  
            
        else :
            vx1 = hv[0,0]
            vy1 = hv[1,0]
            vz1 = hv[2,0]
        
        
        
        VMAX = np.sqrt(vx1*vx1+vy1*vy1+vz1*vz1) 
        
        if (VMAX > vmax):
            vx1 = vx1*vmax/VMAX
            vy1 = vy1*vmax/VMAX
            vz1 = vz1*vmax/VMAX



        V[0,0] = vx1
        V[1,0] = vy1
        V[2,0] = vz1


        try:
            
            
            rob.speedl((vx1 , vy1, vz1 , 0, 0, 0), acc=0.3, min_time=3);
        except:
            print('sent')

        time.sleep(0)

        toc = time.clock()
        print(e)
            
else:
    print('finishing')
    try:
        rob.stopl()
    except:
        print('No')

In [ ]:
xn = 0.1
zn = 0.1
yn = 0.1
tic= 0.1
f= open("guru99.txt","w+")
f.write(str(xn)+";"+str(yn)+";"+str(zn)+";"+str(tic)+"\n")
f.write(str(xn)+";"+str(yn)+";"+str(zn)+";"+str(tic)+"\n")
f.write(str(xn)+";"+str(yn)+";"+str(zn)+";"+str(tic)+"\n")
f.close()

In [ ]:

str(xn)+";"+str(yn)+";"+str(zn)+";"+str(tic)

In [ ]:
with open('data2.json', 'r') as fa:
    data = json.load(fa)
    k1[0,0] = data['controller']['k11']
    k1[1,1] = data['controller']['k12']
    k1[2,2] = data['controller']['k13']
    k2[0,0] = data['controller']['k21']
    k2[1,1] = data['controller']['k22']
    k2[2,2] = data['controller']['k23']
    k3[0,0] = data['controller']['k31']
    k3[1,1] = data['controller']['k32']
    k3[2,2] = data['controller']['k33']
    vmax = data['controller']['vmax']
    vmax2 = data['poss']['vmax']
    x_refr = data['poss']['xref']
    y_refr = data['poss']['yref']
    z_refr = data['poss']['zref']
    xo_refr = data['poss_obs']['xref']
    yo_refr = data['poss_obs']['yref']
    zo_refr = data['poss_obs']['zref']
    dmin = data['poss_obs']['dmin']
    kd1 = data['poss_obs']['kd1']
    kd2 = data['poss_obs']['kd2']
    kd3 = data['poss_obs']['kd3']
    tau = data['poss_obs']['tau']
    kangle = data['poss_obs']['kangle']

In [ ]:
import time
import sys
import urx
import numpy as np
to = time.clock()   
while True:
    tic = time.clock()        
    A = 0.3
    T = 10
    Pa = np.zeros((3,1));   
    tact = tic - to;
    yref = A * np.cos(2*np.pi*(tact)/T)
    zref = A * np.sin(2*np.pi*(tact)/T) + 0.4
    print(yref,zref)
    time.sleep(0.5)

In [7]:
import time
import sys
import urx
import numpy as np
import keyboard
import json
import numpy as np
from numpy.linalg import inv
import time
from urx.robotiq_two_finger_gripper import Robotiq_Two_Finger_Gripper

interval = True

#v =triad_openvr.triad_openvr()

try:
    rob = urx.Robot("192.168.88.240")
except:
    print('robot error')
    interval = False
    
robotiqgrip = Robotiq_Two_Finger_Gripper(rob)
robotiqgrip.close_gripper()
    
#rob.set_freedrive(1)
#time.sleep(0.5)
#rob.speedl((0.1 , 0, 0 , 0, 0, 0), acc=0.3, min_time=1);
#time.sleep(0.5)
#rob.set_freedrive(1)

tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 69
tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 1093
tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 1375
Exception in thread Thread-8:
Traceback (most recent call last):
  File "D:\PROGRAMAS\anaconda\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "D:\PROGRAMAS\anaconda\lib\site-packages\urx\ursecmon.py", line 286, in run
    data = self._get_data()
  File "D:\PROGRAMAS\anaconda\lib\site-packages\urx\ursecmon.py", line 333, in _get_data
    tmp = self._s_secondary.recv(1024)
socket.timeout: timed out

Exception in thread Thread-9:
Traceback (most recent call last):
  File "D:\PROGRAMAS\anaconda\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "D:\PROGRAMAS\anaconda\lib\site-packages\urx\ursecmon.py", line 286, in run
    data = self._get_data()
  File "D:\PROGRAMAS\a

In [ ]:
with open('bt.json', 'r') as fa:
    data = json.load(fa)
    n_points = data['general']['n']
    n_counter = 1
    x_refr = data['position'+str(n_counter)]['xref']
    y_refr = data['position'+str(n_counter)]['yref']
    z_refr = data['position'+str(n_counter)]['zref']

In [ ]:
print(data)

In [4]:
from urx.robotiq_two_finger_gripper import Robotiq_Two_Finger_Gripper
robotiqgrip = Robotiq_Two_Finger_Gripper()

TypeError: __init__() missing 1 required positional argument: 'robot'

Exception in thread Thread-7:
Traceback (most recent call last):
  File "D:\PROGRAMAS\anaconda\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "D:\PROGRAMAS\anaconda\lib\site-packages\urx\ursecmon.py", line 286, in run
    data = self._get_data()
  File "D:\PROGRAMAS\anaconda\lib\site-packages\urx\ursecmon.py", line 333, in _get_data
    tmp = self._s_secondary.recv(1024)
socket.timeout: timed out

Exception in thread Thread-6:
Traceback (most recent call last):
  File "D:\PROGRAMAS\anaconda\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "D:\PROGRAMAS\anaconda\lib\site-packages\urx\ursecmon.py", line 286, in run
    data = self._get_data()
  File "D:\PROGRAMAS\anaconda\lib\site-packages\urx\ursecmon.py", line 333, in _get_data
    tmp = self._s_secondary.recv(1024)
socket.timeout: timed out

